<a href="https://colab.research.google.com/github/USCbiostats/PM520/blob/main/Lab_3_Optimization_PtI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Move on Up, or: Maximum likelihood Estimation & Optimization Pt I

TBD: move notes from slides to here.


In [1]:
import jax
import jax.numpy as jnp
import jax.random as rdm

## MLE for iid Normal data
TBD: Add notes for Normal PDF and MLE estimator

In [5]:
def norm_rv(key, n: int, mu: float, sigma_sq: float):
  """
  Samples $n$ observations from $x_i \sim N(\mu, \sigma^2)$

  n: the number of observations
  mu: the mean parameter
  sigma_sq: the variance parameter

  returns: x, Array of observations
  """
  x = mu + jnp.sqrt(sigma_sq) * rdm.normal(key, shape=(n,))
  return x


def norm_mle(x):
  """
  Computes $\hat{\mu}_{MLE}$ and $\hat{\sigma^2}_{MLE}$.

  x: Array of observations

  returns: Tuple of $\hat{\mu}_{MLE}$ and $\hat{\sigma^2}_{MLE}$.
  """
  mu_hat = pass
  ssq_hat = pass

  return mu_hat, ssq_hat

seed = 0
key = rdm.PRNGKey(seed)
key, x_key = rdm.split(key)

N = 500

mu = 58.
sigma_sq = 100.
x = norm_rv(x_key, N, mu, sigma_sq)
#print(f"x = {x}")
mu_hat, ssq_hat = norm_mle(x)
print(f"MLE[\mu, \sigma^2] = {mu_hat}, {ssq_hat}")

MLE[\mu, \sigma^2] = 58.59890365600586, 98.35617065429688


In [7]:
def sq_diff(param, estimate):
  return (param - estimate) ** 2

mu = 58.
sigma_sq = 10.
for N in [50, 100, 1000, 10000]:
  key, x_key = rdm.split(key)
  # generate N observations
  x_n = norm_rv(x_key, N, mu, sigma_sq)
  # estimate mu, and sigma_sq
  mu_hat, ssq_hat = norm_mle(x_n)
  # compute the sq-diff for both and report
  mu_err = sq_diff(mu, mu_hat)
  ssq_err = sq_diff(sigma_sq, ssq_hat)
  print(f"MSE[{N} | mu, sigma^2] = {mu_err}, {ssq_err}")

MSE[50 | mu, sigma^2] = 1.0336028337478638, 6.931415557861328
MSE[100 | mu, sigma^2] = 0.11436349898576736, 3.418867826461792
MSE[1000 | mu, sigma^2] = 1.0024814400821924e-05, 0.01468171738088131
MSE[10000 | mu, sigma^2] = 0.004498897586017847, 0.015031831339001656


## MLE for iid Exponential data
TBD: Add notes for Exponential PDF and MLE estimator

In [8]:
def exp_rv(key, n: int, rate: float):
  """
  Samples $n$ observations from $x_i \sim Exp(\lambda)$

  n: the number of observations
  rate: the $\lambda$ parameter

  returns: x, Array of observations
  """
  mean = 1 / rate
  x = mean * rdm.exponential(key, shape=(n,))
  return x


def exp_mle(x):
  """
  Computes $\hat{\lambda}_{MLE}$.

  x: Array of observations

  returns: $\hat{\lambda}_{MLE}$.
  """
  rate_hat = pass
  return rate_hat

key, x_key = rdm.split(key)
N = 100
rate = 1 / 500.
x = exp_rv(x_key, N, rate)
print(f"x = {x}")
rate_hat = exp_mle(x)
print(f"MLE[\lambda = {rate}] = {rate_hat}")

x = [4.35397430e+02 4.72187927e+02 1.38809436e+03 2.51057831e+02
 7.86665115e+01 5.10070496e+02 6.66986755e+02 5.30637817e+02
 6.95734985e+02 3.18214294e+02 5.19751167e+01 2.74846916e+01
 3.22565138e-01 4.53188721e+02 6.49053101e+02 6.87411118e+01
 3.77136902e+02 4.52345795e+02 8.62563843e+02 4.71622925e+02
 1.00533356e+03 6.46890991e+02 2.95587494e+02 1.70733911e+03
 8.86031799e+01 3.67736015e+01 1.70877844e+03 1.35494446e+02
 6.13494530e+01 6.64146118e+02 2.19395728e+03 1.05528516e+03
 9.19410034e+02 3.70226440e+01 2.18483673e+02 5.09372223e+02
 2.02266724e+02 4.67761505e+02 1.08360504e+02 2.41796265e+03
 4.80211212e+02 9.36368164e+02 5.30847473e+02 2.47510696e+02
 3.98499237e+02 2.79448181e+02 1.04075806e+03 9.33896729e+02
 3.96697357e+02 4.65119263e+02 7.25767761e+02 9.85023422e+01
 5.21523010e+02 3.69823975e+02 1.16978165e+02 2.81428162e+02
 1.20678604e+02 1.87231949e+02 1.82771103e+02 9.60759338e+02
 7.37772095e+02 6.62937164e+01 3.19520782e+02 2.58933685e+02
 1.22909531e+02 1.70

In [9]:
rate = 1 / 50.
for N in [50, 100, 1000, 10000]:
  key, x_key = rdm.split(key)
  # generate N observations
  x_n = exp_rv(x_key, N, rate)
  # estimate rate
  rate_hat = exp_mle(x_n)
  # compute the sq-diff for rate
  rate_err = sq_diff(rate, rate_hat)
  print(f"MSE[{N} | \lambda = {rate}] = {rate_err}")

MSE[50 | \lambda = 0.02] = 9.133363164437469e-06
MSE[100 | \lambda = 0.02] = 6.241853043320589e-07
MSE[1000 | \lambda = 0.02] = 2.641813523496239e-07
MSE[10000 | \lambda = 0.02] = 4.547181120528876e-08


# Gradient descent
TBD: Add notes for gradient descent

In [ ]:
def sim_linear_reg(key, N, P, r2=0.5):
  key, x_key = rdm.split(key)
  X = rdm.normal(x_key, shape=(N, P))

  key, b_key = rdm.split(key)
  beta = rdm.normal(b_key, shape=(P,))

  # g = jnp.dot(X, beta)
  g = X @ beta
  s2g = jnp.var(g)

  # back out what s2e is, such that s2g / (s2g + s2e) == h2
  s2e = (1 - r2) / r2 * s2g
  key, y_key = rdm.split(key)

  # add env noise to g, but scale such that var(e) == s2e
  y = g + jnp.sqrt(s2e) * rdm.normal(y_key, shape=(N,))
  return y, X, beta

key, sim_key = rdm.split(key)

N = 1000
P = 5
y, X, beta = sim_linear_reg(sim_key, N, P)

def linreg_loss(y, X, beta_hat):
  pass

def gradient(y, X, beta_hat):
  pass

step_size = 1 / N
diff = 10.
last_loss = 1000.
idx = 0
beta_hat = jnp.zeros((P,))
# while delta in loss is large, continue
while jnp.fabs(diff) > 1e-3:
  # take a step in the direction of the gradient using step_size
  beta_hat = beta_hat - step_size * gradient(y, X, beta_hat)
  # update our current loss and compute delta
  cur_loss = linreg_loss(y, X, beta_hat)
  diff = last_loss - cur_loss
  last_loss = cur_loss
  # wave to the crowd
  print(f"Loss[{idx} | {beta}] = {last_loss} @ {beta_hat}")
  idx += 1

Loss[0 | [-1.0257084   1.8594663   1.1235882   0.23114179 -0.34901348]] = 5639.55810546875 @ [-1.079702    1.8402483   1.1758611   0.23298487 -0.52076465]
Loss[1 | [-1.0257084   1.8594663   1.1235882   0.23114179 -0.34901348]] = 5623.48291015625 @ [-1.1577635   1.8372455   1.081856    0.24313869 -0.47761846]
Loss[2 | [-1.0257084   1.8594663   1.1235882   0.23114179 -0.34901348]] = 5623.23828125 @ [-1.1617126  1.8369826  1.0963572  0.2397221 -0.4716044]
Loss[3 | [-1.0257084   1.8594663   1.1235882   0.23114179 -0.34901348]] = 5623.23193359375 @ [-1.1620235   1.8380398   1.094032    0.23982307 -0.47083738]
Loss[4 | [-1.0257084   1.8594663   1.1235882   0.23114179 -0.34901348]] = 5623.2314453125 @ [-1.1620889   1.8379307   1.0944436   0.23976085 -0.4707122 ]


In [ ]:
key, sim_key = rdm.split(key)

N = 1000
P = 5
y, X, beta = sim_linear_reg(sim_key, N, P)

step_size = 1 / N
diff = 10.
last_loss = 1000.
idx = 0
beta_hat = jnp.zeros((P,))
# while delta in loss is large, continue
while jnp.fabs(diff) > 1e-3:
  # take a step in the direction of the gradient using step_size
  beta_hat = beta_hat - step_size * # COMPUTE GRADIENT USING JAX
  # update our current loss and compute delta
  cur_loss = linreg_loss(y, X, beta_hat)
  diff = last_loss - cur_loss
  last_loss = cur_loss
  # wave to the crowd
  print(f"Loss[{idx} | {beta}] = {last_loss} @ {beta_hat}")
  idx += 1